# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Import Data

Upload User Data 

In [ ]:
users_path = ('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/msd_user_data.csv')

users = pd.read_csv(users_path, na_values = 'NaN', index_col = 0)

users.index.names = ['index']

print(users.info())

users.head()

Upload Metadata Data

In [ ]:
songs_path = ('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/msd_song_metadata.csv')

songs = pd.read_csv(songs_path, na_values = 'NaN', index_col = 0)

songs.index.names = ['index']

print(songs.info())

songs.head()

Upload Artist Tags

In [ ]:
# Upload Artist Tags 

tags_path = ('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/msd_tags.csv')

tags_data = pd.read_csv(tags_path, index_col = 0, na_values = 'NaN',
                        dtype = {0:'int', 1:'str',2:'str',3:'str'})

tags_data.index.names = ['index']

# Filter User Data

+ Remove users with less than five different song listens.
+ Remove users with more than 1,750 song listens.

**Remove and save users with less than five different song interactions**

In [ ]:
threshold_count = 5

c = Counter(users.user_id)

relevant_items = [k for k, count in c.iteritems() if count > threshold_count]

In [ ]:
# Least Interactions

Counter(c).most_common()[-1]

# The least interactions is 10. So at threshold 5, no users are removed.

In [ ]:
users_filtered = users.loc[users.user_id.isin(relevant_items), :]

users_filtered.info()

**Remove users with more than 1,750 song listens**

In [ ]:
#Average song duration in minutes

print('Average Song Duration:', np.mean(songs.duration)/60)

# Listens an user can do in 24 hrs without interruptions. 

min_day = 60*24 # minutes in a day

days = 10 #days of user listening represented in the dataset

print('Max User Listening Per Day Estimate:', min_day/(np.mean(songs.duration)/60))

# Listens an user can do over ten days WITHOUT INTERRUPTION

days = 10 #days of user listening represented in the dataset

print('Max User Un-interrupted Listening in Ten Days Estimate:', days*(min_day/(np.mean(songs.duration)/60)))


In my opinion, users with more than 12hrs of un-interrupted listening over 10 days should be investigated further.


These will be users with more than 1,750 listens. It is highly unlikely a 'typical' user will behave this way, also it is physically unlikely. I will eliminate these users from the dataset.

In [ ]:
users_grouped = users.groupby('user_id').sum()

users_grouped = users_grouped[users_grouped['play_count'] > 1749]

users_grouped.info()

In [ ]:
users_filtered[users_filtered['user_id'] != users_grouped['user_id']]

# Create User-Song Matrix

Merge song and user data to create 'relationships' between users and items.

In [ ]:
%time

#Merge user matrix with song matrix. Use keys from left df to match.

user_song_matrix = songs.merge(users, on='song_id', how = 'outer')

#Based in exploratory analysis, some songs will have no interactions at all.

Replace missing values with zeroes in the play count and play.

In [ ]:
#Replace missing values with zeroes in the play indicator and play counts.

user_song_matrix[['play','play_count']] = user_song_matrix[['play','play_count']].fillna(0)

#Reset index

user_song_matrix.reset_index()

user_song_matrix.head()

# Drop Unneded Columns

In [ ]:
user_song_matrix.info()

In [ ]:
user_song_matrix.drop(labels = ['artist_7digitalid','artist_latitude','artist_location','artist_longitude',
                                'artist_mbid','artist_playmeid','idx_artist_terms','idx_similar_artists',
                                'release_7digitalid','track_7digitalid','analysis_sample_rate',
                                'audio_md5','danceability','energy','end_of_fade_in','idx_bars_confidence',
                                'idx_bars_start','idx_beats_confidence','idx_sections_start',
                                'idx_segments_confidence','idx_segments_loudness_max',
                                'idx_segments_loudness_max_time','idx_segments_loudness_start',
                                'idx_segments_pitches','idx_segments_start','idx_segments_timbre',
                                'idx_tatums_confidence','idx_tatums_start','key_confidence','mode_confidence',
                                'start_of_fade_out','time_signature_confidence','track_id','idx_artist_mbtags',
                                'idx_beats_start','idx_sections_confidence'], axis = 1,inplace = True)

# Randomly shuffle data

The data comes ordered by song and users. Shuffle this order.

In [ ]:
user_song_matrix = user_song_matrix.sample(frac=1).reset_index(drop=True)

# Output User - Song Matrix to CSV